# Imports and initialization

In [ ]:
%matplotlib inline
import SQGfunctions as fn
from SQGfunctions import *

## Initialize collapse operators and get drive frequency

In [ ]:
#collapse operators
c_op1 = []
c_op2 = []

#energies and sweet spot defined in SQGfunctions.py file
H_sys,evals,evecs = fluxonium_system(sweet_spot,Ej,Ec,El,n_op,phi_op,H_lc,H_i)
Es = np.real(qubit_energies(H_sys,evals,evecs))
omega_drive = (Es[2]-Es[0])
drive_freq = omega_drive #define drive frequency

# Optimization

## Loop over gate time

In [ ]:
fids = []
errors = []
lambdas = [] #DRAG scaling factor
Eds = [] #drive amplitude

bnds = ((-3, 3), (0, 2)) #bounds, upper bound for drive amplitude estimated from analytic calculation
x_start = [0.5,1.5]
sigma_start = 0.5

times = np.linspace(10,50,41)
for t in times:
    optfunc = lambda x: 1-gate_pulse_no_fp(x[0],x[1],gatetime=t,drivefrequency=drive_freq,psi=initRho)[0]
    opts = cma.CMAOptions()
    #bounds for drive strength estimated from analytic calculation
    if t <= 20:
        x_start = [0.5, 1.5]
        bnds = ((-3, 3), (0, 2))
        opts.set("bounds", [(bnds[0][0], bnds[1][0]),(bnds[0][1], bnds[1][1])])
    elif t <= 30:
        x_start = [0.5, 0.75]
        bnds = ((-3, 3), (0, 1))
        opts.set("bounds", [(bnds[0][0], bnds[1][0]),(bnds[0][1], bnds[1][1])])
    elif t <= 40:
        x_start = [0.5, 0.5]
        bnds = ((-3, 3), (0, 0.7))
        opts.set("bounds", [(bnds[0][0], bnds[1][0]),(bnds[0][1], bnds[1][1])])
    else:
        x_start = [0.5, 0.25]
        bnds = ((-3, 3), (0, 0.5))
        opts.set("bounds", [(bnds[0][0], bnds[1][0]),(bnds[0][1], bnds[1][1])]) 
    opts.set('maxfevals', 1000)
    result = cma.fmin(optfunc, x0=x_start, sigma0=sigma_start, options=opts)
    result_best = vars(result[-2].best)
    values = result_best.get('x')
    error = result_best.get('f')
    errors.append(error)
    fids.append(1-error)
    lambdas.append(values[0])
    Eds.append(values[1])

# Export Data

In [ ]:
data = {'FidelityCMA': fids,
        'ErrorCMA': errors,
        'LambdasCMA': lambdas,
        'EdsCMA': Eds}

df = pd.DataFrame(data)
df.to_csv('\filepath\filename.csv')